## ExCAPE-DB

Explore the ExCAPE dataset. Processing tsv file with pandas is slow. We can consider convert the dataset to sqlite if we need to do more heavy analysis.

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from random import sample
from json import load, dump
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv("./resource/pubchem.chembl.dataset4publication_inchi_smiles.tsv", sep='\t')

In [196]:
print(df.shape)
df.head()

(70850163, 12)


,Ambit_InchiKey,Original_Entry_ID,Entrez_ID,Activity_Flag,pXC50,DB,Original_Assay_ID,Tax_ID,Gene_Symbol,Ortholog_Group,InChI,SMILES
0,AAAAZQPHATYWOK-YRBRRWAQNA-N,11399331,2064,A,7.19382,pubchem,248914,9606,ERBB2,1346,InChI=1/C32H29ClN6O3S/c1-4-41-28-16-25-22(15-2...,ClC=1C=C(NC=2C=3C(N=CC2C#N)=CC(OCC)=C(NC(=O)/C...
1,AAAAZQPHATYWOK-YRBRRWAQNA-N,CHEMBL175513,1956,A,6.73000,chembl20,312997,9606,EGFR,1260,InChI=1/C32H29ClN6O3S/c1-4-41-28-16-25-22(15-2...,C1=2C(=C(C#N)C=NC1=CC(=C(C2)NC(/C=C/CN(C)C)=O)...
2,AAABHMIRDIOYOK-NPVYFSBINA-N,CHEMBL1527551,10919,N,4.55000,chembl20,737344,9606,EHMT2,6822,InChI=1/C18H14N6O3/c1-23-10-15(24(26)27)16(22-...,O=C(NC=1C=C2N=C(NC2=CC1)C=3C=CC=CC3)C4=NN(C=C4...
3,AAABHMIRDIOYOK-NPVYFSBINA-N,CHEMBL1527551,19885,A,5.35000,chembl20,688759,10090,RORC,3770,InChI=1/C18H14N6O3/c1-23-10-15(24(26)27)16(22-...,O=C(NC=1C=C2N=C(NC2=CC1)C=3C=CC=CC3)C4=NN(C=C4...
4,AAABHMIRDIOYOK-NPVYFSBINA-N,CHEMBL1527551,216,N,4.40000,chembl20,688238,9606,ALDH1A1,143,InChI=1/C18H14N6O3/c1-23-10-15(24(26)27)16(22-...,O=C(NC=1C=C2N=C(NC2=CC1)C=3C=CC=CC3)C4=NN(C=C4...


In [24]:
assay_ids = df["Original_Assay_ID"].tolist()
len(set(assay_ids))

85307

In [25]:
target_ids = df["Entrez_ID"].tolist()
len(set(target_ids))

1667

In [26]:
Counter(df["Activity_Flag"])

Counter({'A': 1332426, 'N': 69517737})

## Check Overlapping with InChI

In [2]:
bone_df = pd.read_csv("./data/test/meta_data/chemical_annotations_inchi.csv")
bone_df.head()

,BROAD_ID,CPD_NAME,CPD_NAME_TYPE,CPD_SAMPLE_ID,DOS_LIBRARY,SOURCE_NAME,CHEMIST_NAME,VENDOR_CATALOG_ID,CPD_SMILES,USERCOMMENT,CPD_CANONICAL_SMILES,INCHI
0,BRD-A56675431-001-04-0,altizide,INN,SA82748,NaN,Prestwick Chemical Inc.,NaN,Prestw-721,NS(=O)(=O)c1cc2c(NC(CSCC=C)NS2(=O)=O)cc1Cl,NaN,C=CCSCC1Nc2cc(Cl)c(S(N)(=O)=O)cc2S(=O)(=O)N1,InChI=1S/C11H14ClN3O4S3/c1-2-3-20-6-11-14-8-4-...
1,BRD-A51829654-001-01-4,"BRL-15,572",common,SA82481,NaN,Biomol International Inc.,NaN,AC-536,OC(CN1CCN(CC1)c1cccc(Cl)c1)C(c1ccccc1)c1ccccc1,NaN,OC(CN1CCN(c2cccc(Cl)c2)CC1)C(c1ccccc1)c1ccccc1,InChI=1S/C25H27ClN2O/c26-22-12-7-13-23(18-22)2...
2,BRD-K04046242-001-03-6,equilin,primary-common,SA82922,NaN,Prestwick Chemical Inc.,NaN,Prestw-850,C[C@]12CC[C@H]3C(=CCc4cc(O)ccc34)[C@@H]1CCC2=O,NaN,CC12CCC3C(=CCc4cc(O)ccc43)C1CCC2=O,InChI=1S/C18H20O2/c1-18-9-8-14-13-5-3-12(19)10...
3,BRD-K16508793-001-01-8,diazepam,INN,SA59660,NaN,MicroSource Discovery Systems Inc.,NaN,1900003,CN1c2ccc(Cl)cc2C(=NCC1=O)c1ccccc1,NaN,CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...
4,BRD-K09397065-001-01-6,SR 57227A,to-be-curated,SA82504,NaN,Biomol International Inc.,NaN,AC-561,NC1CCN(CC1)c1cccc(Cl)n1,NaN,NC1CCN(c2cccc(Cl)n2)CC1,InChI=1S/C10H14ClN3/c11-9-2-1-3-10(13-9)14-6-4...


In [8]:
total_inchi = set(df["InChI"])
bone_inchi = set(bone_df["INCHI"])
print("There are {} compounds in U2OS, and {} compounds in ExCAPE.".format(len(bone_inchi),
                                                                           len(total_inchi)))

There are 30405 compounds in U2OS, and 998535 compounds in ExCAPE.


In [32]:
total_inchi & bone_inchi

set()

In [42]:
sample(bone_inchi, 10)

['InChI=1S/C22H18N2O5S2/c1-10-3-6-12(7-4-10)24-20(26)16-15(11-5-8-13(25)14(9-11)29-2)17-19(23-22(28)31-17)30-18(16)21(24)27/h3-9,15-16,18,25H,1-2H3,(H,23,28)',
 'InChI=1S/C15H15N/c1-2-6-12(7-3-1)15-11-16-10-13-8-4-5-9-14(13)15/h1-9,15-16H,10-11H2',
 'InChI=1S/C28H34FN3O5/c1-17-14-32(28(35)21-7-5-6-8-23(21)29)18(2)16-37-24-12-11-20(30-26(33)19-9-10-19)13-22(24)27(34)31(3)15-25(17)36-4/h5-8,11-13,17-19,25H,9-10,14-16H2,1-4H3,(H,30,33)/t17-,18-,25+/m1/s1',
 'InChI=1S/C17H16FNO3/c18-14-4-2-1-3-13(14)17(20)19-8-7-12-5-6-15-16(11-12)22-10-9-21-15/h1-6,11H,7-10H2,(H,19,20)',
 'InChI=1S/C10H10O4/c1-14-10(13)5-2-7-6-8(11)3-4-9(7)12/h2-6,11-12H,1H3',
 'InChI=1S/C25H23N5O4/c1-15-2-4-17(5-3-15)22-21-20(19(14-26)23(27)34-24(21)29-28-22)16-6-8-18(9-7-16)33-25(31)30-10-12-32-13-11-30/h2-9,20H,10-13,27H2,1H3,(H,28,29)',
 'InChI=1S/C31H42N4O6/c1-20-17-35(21(2)19-36)30(38)25-11-8-12-26(33-29(37)22-9-6-5-7-10-22)28(25)41-27(20)18-34(3)31(39)32-23-13-15-24(40-4)16-14-23/h8,11-16,20-22,27,36H,5-7,9-10,17-1

In [43]:
sample(total_inchi, 10)

['InChI=1/C23H14ClF2N3O/c24-18-11-15(9-10-19(18)26)27-23-28-21(13-5-7-14(25)8-6-13)17-12-30-20-4-2-1-3-16(20)22(17)29-23/h1-11H,12H2,(H,27,28,29)/f/h27H',
 'InChI=1/C20H21N3O5S/c1-22-17-7-6-16(12-18(17)28-20(22)25)29(26,27)21-10-8-19(24)23-11-9-14-4-2-3-5-15(14)13-23/h2-7,12,21H,8-11,13H2,1H3',
 'InChI=1/C26H27N3O8S/c1-16-11-22(17(2)29(16)20-7-10-24-25(12-20)37-15-36-24)23(31)14-35-26(32)13-28(4)38(33,34)21-8-5-19(6-9-21)27-18(3)30/h5-12H,13-15H2,1-4H3,(H,27,30)/f/h27H',
 'InChI=1/C20H25F3N4O2/c21-20(22,23)16-6-4-15(5-7-16)19-24-18(29-25-19)14-27-8-2-1-3-17(27)13-26-9-11-28-12-10-26/h4-7,17H,1-3,8-14H2',
 'InChI=1/C20H16Cl3N3O2/c21-11-1-2-12(16(22)6-11)14-8-24-9-15(14)20(28)26-18-5-10-3-4-25-19(27)13(10)7-17(18)23/h1-7,14-15,24H,8-9H2,(H,25,27)(H,26,28)/f/h26-27H',
 'InChI=1/C22H20N2O4/c1-27-19-10-4-8-17(13-19)23-21(25)15-6-3-7-16(12-15)22(26)24-18-9-5-11-20(14-18)28-2/h3-14H,1-2H3,(H,23,25)(H,24,26)/f/h23-24H',
 'InChI=1/C20H29N3O3/c1-15-7-5-9-18(16(15)2)26-14-11-21-10-3-4-12-23-19(24

There are no overlapping compounds. After inspecting on the random 10 samples from each list, we notice the `rdkit` converted InChI is in standard format (`InChI=1S`) while ExCAPE-DB is using version 1 (`InchI=1`) .

In [65]:
modified_bone_inchi = set([re.sub(r'(InChI=1S)(.+)', r'InChI=1\2', s) for s in bone_inchi if not pd.isna(s)])
len(total_inchi & modified_bone_inchi)

9468

After naively remove "S" from our InChI strings, we got 9468 intersections with ExCAPE-DB. In he openReview paper, they got 11,585 interactions.

The difference can be explained by:
1. We are using a cleaned dataset instead of raw ChEMBL.
2. It is a wrong way to convert version 1 InChI to standard InChI.

## Check Overlapping with Converted InChI

ExCAPE-DB also provides SMILES encoding. Hopefully an intersection with SMILES gives the same result as above. We are not sure if the encoding is the same as the canonical SMILES generated by `rdkit`. We can try to intersect with the canonical SMILES from U2OS first.

In [66]:
total_smiles = set(df["SMILES"])
bone_smiles = set(bone_df["CPD_CANONICAL_SMILES"])

In [67]:
total_smiles & bone_smiles

set()

It seems the the SMILES from ExCAPE-DB are not canonical. We can use Condor to convert all InChI entries from ExCAPE-DB to the standard InChI format, and compute the overlapping again.

In [6]:
converted_inchi = load(open("./resource/std_inchi.json", 'r'))
len(set(converted_inchi))

984413

The number of compounds in ExCAPE-DB drops after conversion, since there are some entries `rdkit` fail to convert. 

In [7]:
len(set(converted_inchi) & set(bone_df["INCHI"]))

27000

The number of overlapping compounds increases from 9468 to 27000. It shows that changing '1' to `1S` manually is wrong. However, `rdkit` gives warning `Omitted undefined stereo` and/or `Charges were rearranged` for 258831 compounds during conversion. Due to the high number overlapping compounds, theses two warnings may not directly lead to accidental overlapping with U2OS InChI, but we still want to understand the warnings before committing to this method.

## Thresholding Assays

According to the [ICLR paper](https://openreview.net/pdf?id=S1gBgnR9Y7), we want to further filter the assays to use.

Some filtering ideas from the paper:
1. Use assays having results from at least 10 different compounds.
1. Use assays having at least 10 positive and 10 negative samples (stronger).
2. Multi-threshold concentration response (CR) on positive results at level 5.5, 6.5, 7.5, then concatenate three matrices (how?).

**Note: The following filtering are using the 9,468 overlapping compounds instead of 27,000.**

In [82]:
intersection = total_inchi & modified_bone_inchi

# Construct a subset dataframe with the intersection compounds
intersect_index = [True if i in intersection else False for i in df["InChI"]]
intersect_df = df[intersect_index]
intersect_df.shape

(849383, 12)

#### Filter 1: At least 10 different compounds

In [87]:
# Filtering targets with at least 10 compounds
target_count = Counter(intersect_df["Entrez_ID"])
filter_1_targets = [t for t in target_count if target_count[t] >= 10]

print("There are {} targets for those {} compounds, and {} of the assays have at least 10 compounds tested.".\
      format(len(target_count), len(intersection), len(aval_targets)))

There are 945 targets for those 9468 compounds, and 575 of the assays have at least 10 compounds tested.


#### Filter 2: At least 10 positive and 10 negative samples

In [115]:
# Filtering targets with at least 10 positive results and 10 negative results
pos_intersect = intersect_df[intersect_df["Activity_Flag"] != "N"]
pos_count = Counter(pos_intersect["Entrez_ID"])

neg_intersect = intersect_df[intersect_df["Activity_Flag"] == "N"]
neg_count = Counter(neg_intersect["Entrez_ID"])

filter_2_targets = [t for t in target_count if (t in neg_count) and (neg_count[t] >= 10) and
                    (t in pos_count) and (pos_count[t] >= 10)]

print("There are {} positive results and {} negative results. {} assays haivng both results greater or equal to 10.".\
      format(len(pos_intersect), len(neg_intersect), len(filter_2_targets)))

There are 19462 positive results and 829921 negative results. 141 assays haivng both results greater or equal to 10.


In [199]:
filter_2_df = intersect_df[intersect_df["Entrez_ID"].isin(filter_2_targets)]
filter_2_compounds = set(filter_2_df["InChI"])

print("Under filter 2, there are {} compounds left.".format(len(filter_2_compounds)))

Under filter 2, there are 9404 compounds left.


#### Filter 3: Concentration response thresholds

In [145]:
dump(pos_intersect["pXC50"].tolist(), open("./plots/xc_all.json", 'w'), indent=2)
filter_2_index = [True if t in filter_2_targets else False for t in pos_intersect["Entrez_ID"]]
dump(pos_intersect[filter_2_index]["pXC50"].tolist(), open("./plots/xc_filter_2.json", 'w'), indent=2)

![](./plots/pxc50.png)

Here is the plot of the `pxc50` distribution. Higher value indicates the higher confidence for positive effect. According to the paper of ExCAPE, they should have removed all samples with `pxc50 > 10`.

#### Comments

1. For now, I think filter 2 alone is good enough. We can get a $9404 \times 141$ output matrix.
2. We can discuss how to concatenate three matrices with different `pxc50` thresholds.

In [159]:
filter_2_df = intersect_df[intersect_df["Entrez_ID"].isin(filter_2_targets)]
print(filter_2_df.shape)

# Save the new csv
intersect_df.to_csv("intersect_df.csv", index=False)
filter_2_df.to_csv("filtered_df.csv", index=False)

(357796, 12)


In [9]:
# Save the selected compound identifier
filter_2_compounds = set(filter_2_df["InChI"])
board_ids = []
inchis = []

for c in filter_2_compounds:
    # Convert InChI v1 to standard InChI
    std_name = re.sub(r'(InChI=1)(.+)', r'InChI=1S\2', c)
    board_id = bone_df.loc[bone_df["INCHI"] == std_name]["BROAD_ID"].values[0]
    board_ids.append(board_id)
    
    # We also want to collect inchis, because the order of set() is unstable
    # We want to keep the board_ids and inchis have the same order
    # Collect the v1 InChI here
    inchis.append(c)
    
print(len(board_ids))
dump({"board_ids": board_ids,
      "inchis": inchis},
     open("./resource/selected_compounds.json", 'w'), indent=2)

9404


## Build Output Matrix

Let's build our multi-task output matrix with size $9404\times 141$.

In [3]:
filter_2_df = pd.read_csv("./resource/filtered_df.csv")
filter_2_df.head()

,Ambit_InchiKey,Original_Entry_ID,Entrez_ID,Activity_Flag,pXC50,DB,Original_Assay_ID,Tax_ID,Gene_Symbol,Ortholog_Group,InChI,SMILES
0,AACRWZVDRSTLKY-UHFFFAOYNA-N,796971,1081,N,NaN,pubchem,624291,9606,CGA,731,InChI=1/C11H14N2O4S/c14-13(15)10-6-2-3-7-11(10...,S(=O)(=O)(N1CCCCC1)C=2C(N(=O)=O)=CC=CC2
1,AACRWZVDRSTLKY-UHFFFAOYNA-N,796971,10951,N,NaN,pubchem,540317,9606,CBX1,6848,InChI=1/C11H14N2O4S/c14-13(15)10-6-2-3-7-11(10...,S(=O)(=O)(N1CCCCC1)C=2C(N(=O)=O)=CC=CC2
2,AACRWZVDRSTLKY-UHFFFAOYNA-N,796971,11201,N,NaN,pubchem,588590,9606,POLI,7040,InChI=1/C11H14N2O4S/c14-13(15)10-6-2-3-7-11(10...,S(=O)(=O)(N1CCCCC1)C=2C(N(=O)=O)=CC=CC2
3,AACRWZVDRSTLKY-UHFFFAOYNA-N,796971,1128,N,NaN,pubchem_screening,588852,9606,CHRM1,752,InChI=1/C11H14N2O4S/c14-13(15)10-6-2-3-7-11(10...,S(=O)(=O)(N1CCCCC1)C=2C(N(=O)=O)=CC=CC2
4,AACRWZVDRSTLKY-UHFFFAOYNA-N,796971,1132,N,NaN,pubchem_screening,624127,9606,CHRM4,756,InChI=1/C11H14N2O4S/c14-13(15)10-6-2-3-7-11(10...,S(=O)(=O)(N1CCCCC1)C=2C(N(=O)=O)=CC=CC2


In [29]:
# The schema we use: -1: NA, 0: Neg, 1: Pos
output_matrix = np.zeros((9404, 141)) - 1

# Create the order dictionary for all selected comounds
selected_compounds = load(open("./resource/selected_compounds.json", 'r'))["inchis"]
inchi_order_dict = {}
for i in range(len(selected_compounds)):
    inchi_order_dict[selected_compounds[i]] = i
    
# Create the order dictionary for all assays
assay_order_dict = {}
count = 0
for i in set(filter_2_df["Entrez_ID"]):
    assay_order_dict[i] = count
    count += 1
print(count)

141


In [30]:
# The most efficient way to build this matrix is to iterate through the rows of filter_2_df
# while filling in the pre-inited matrx

for index, row in filter_2_df.iterrows():
    cur_inchi = row["InChI"]
    cur_assay = row["Entrez_ID"]
    output_matrix[inchi_order_dict[cur_inchi], assay_order_dict[cur_assay]] = 0 if \
        row["Activity_Flag"] == 'N' else 1

In [31]:
output_matrix

array([[-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1.,  1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ...,  0., -1., -1.]])

In [49]:
count = Counter(output_matrix.reshape(-1).tolist())
total = output_matrix.shape[0] * output_matrix.shape[1]
print("In our output matrix, there are {:.2f}% NA, {:.2f}% Positive, and {:.2f}% Negative.".format(
    count[-1]/total*100, count[1]/total*100, count[0]/total*100))
count

In our output matrix, there are 73.02% NA, 1.06% Positive, and 25.92% Negative.


Counter({-1.0: 968221, 0.0: 343664, 1.0: 14079})

In [50]:
# Save the output matrix along with row/column label info
np.savez("./resource/output_matrix.npz", output_matrix=output_matrix,
         compound_order=inchi_order_dict, assay_order=assay_order_dict)